In [42]:
import pandas as pd


original = pd.read_csv('data.csv')
original = original.drop(['都道府県名ローマ字', '市区町村名ローマ字', '大字町丁目コード', '大字町丁目名', '緯度', '経度'], axis=1)
original = original.rename(columns={'都道府県コード':  'prefecture_code', 
                                                                 '都道府県名': ' prefecture_name',
                                                                 '都道府県名カナ': 'prefecture_name_ruby', 
                                                                 '市区町村コード': 'city_code',
                                                                 '市区町村名': 'city_name',
                                                                 '市区町村名カナ': 'city_name_ruby',})

station = pd.read_csv('station.csv')
station = station.drop(['station_name_k', 'station_name_r', 'post', 'lon', 'lat', 'open_ymd', 'close_ymd', 'e_status', 'e_sort'], axis=1)
station = station.rename(columns={'station_cd': 'station_code',
                                                              'station_g_cd': 'station_g_code',
                                                              'line_cd': 'line_code',
                                                              'pref_cd': 'prefecture_code'})



prefecture = original.iloc[:,:3]
counter_prefecture = prefecture.iloc[0, 0]
drop_prefecture = []
for i in range(len(prefecture)):
    if prefecture.iloc[i, 0] == counter_prefecture:
        drop_prefecture.append(i)
    else:
        counter_prefecture = prefecture.iloc[i, 0]
        continue
prefecture = prefecture.drop(drop_prefecture)

city = original.drop(columns=original.columns[[1, 2]])
counter_city = city.iloc[0,1]
drop_city = []
for i in range(len(city)):
    if city.iloc[i, 1] == counter_city:
        drop_city.append(i)
    else:
        counter_city = city.iloc[i, 1]
        continue
city =  city.drop(drop_city)

print(city)
# append_city_code_list = []
# for i in range(len(station)):
#     for j in range(len(city)):
#         if station.iloc[i,5] in city.iloc[j, 2]:
#             append_city_code_list.append(city.iloc[j, 2])
#             continue
            
# station['city_code'] = append_city_code_list
# print(len(station))
print(len(append_city_code_list))


10855
32
